In [7]:
!pip install langchain langchain-community jq faiss-gpu sentence-transformers langchain-core

  Using cached jq-1.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.4 kB)
  Using cached sentence_transformers-2.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached torch-2.2.2-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2024.3.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x

In [1]:
from langchain_community.chat_models import BedrockChat
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import JSONLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory.buffer import ConversationBufferMemory
from langchain.chains.question_answering import load_qa_chain
import boto3
import os

In [3]:
# setting model ID -- can be found in bedrock 
## note: we are using a chat model
modelID = 'anthropic.claude-3-haiku-20240307-v1:0'

# creating client object to talk to models in bedrock using API
bedrock_client = boto3.client(
    service_name = "bedrock-runtime",
    region_name = "us-east-1"
)

# loading claude 3 haiku using bedrock client 
model = BedrockChat(
    model_id=modelID,
    client=bedrock_client
)

In [4]:
# this function loads the json data 
def load_data(json_path):
    loader = JSONLoader(file_path=json_path, jq_schema=".[]", text_content=False)
    documents = loader.load()
    return documents

# this function loads the json data into a vector database after using the all-mpnet-base-v2 model to embed the text         
def ss_search(documents, user_query):
    embedding_model = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")
    db = FAISS.from_documents(documents, embedding_model)
    docs = db.similarity_search(user_query, k = 15)
    return docs

In [8]:
# decalre json path & run function with json path
json_path = "./ARXIV/merged.json"
documents = load_data(json_path = json_path)

def algoscholar_chat(user_query, documents):
    # complete similarity search between user query and vector space to identify which texts are most similar to the question
    context = ss_search(documents = documents, user_query = user_query)
    
    template = """
    You are an intelligent and helpful research assistant. Your name is AlgoScholar. You will work with the user to help them learn about any research topic or subject area they specify by using the context provided. 

    Here is a list of publications you should reference when answering the user: <arxiv> {context} </arxiv>

    Here are some important rules for the interaction:
    - Always stay in character as AlgoScholar, an AI Assistant from Fannie Mae 
    - If you are unsure how to respond, say 'Sorry, I didn't understand that. Could you repeat the question?'
    - If someone asks something irrelevant, say, 'Sorry, I am AlgoScholar and I help users identify academic publications for a certain topic. Do you have a reearch question today I can help you with?

    Here is an example of how to respond in a standard interaction:
    <example> 
    User: Hi, how were you created and what do you do?
    AlgoScholar: Hello! My name is AlgoScholar, and I was created by a team of technical folks to aid in the identification of academic research papers that pertain to a certain subject. What can I help you with today?
    </example>
    
    Here is the conversation history (between the user and you) prior to the question. It could be empty if there is no history:
    <history> {chat_history} </history>
    
    Here is the user's question: <question> {user_query} </question>

    How do you respond to the user's question? You will analyze the context provided, find the top three most relevant papers per the subject matter provided by the user, and generate a summary of the publication based on the abstract with the url included.

    Think about your answer before you respond. Put your response in <response></response> tags. 
    """

    #put template into PromptTemplate format to work wiht other langchain functions -- filling in the template with the dynamic variables (e.g., history, query, and context)
    prompt = PromptTemplate(
        input_variables=["chat_history", "user_query", "context"], 
        template=template
    )

    # create memory object using key identifiers in prompt 
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        input_key="user_query"
    )

    # create langchain chain
    chain = load_qa_chain(
        model, chain_type="stuff", memory=memory, prompt=prompt
    )
    
    response = chain({"input_documents": context, "user_query": user_query}, return_only_outputs=True)
    print(response['output_text'])

In [9]:
user_query = "Help me understand mechanisms to measure the amount of competition that exists"

algoscholar_chat(user_query = user_query, documents = documents)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


<response>
Based on the user's request to understand mechanisms to measure the amount of competition that exists, the following three publications from the provided list appear most relevant:

1. "International Trade Network: Country Centrality and COVID-19 Pandemic" (arxiv.org/abs/2107.14554)
This paper examines how the topology of the global trade network can explain the rate of COVID-19 diffusion and mortality across countries. The authors compute centrality measures of countries in the trade network and use these to analyze how a country's position in the network impacts the number of infections and fatalities.

2. "Strategic Energy Flows in Input-Output Relations: A Temporal Multilayer Approach" (arxiv.org/abs/2212.11585)
This work constructs a directed and weighted temporal multilayer network to analyze the flow of renewable and non-renewable energy sources between sectors and countries over time. The authors propose methods to identify critical sectors and economies in the syste